# 📌 기상청 api 데이터 전처리 

In [72]:
import numpy as np
import pandas as pd

### 1.데이터 불러오기

In [117]:
asos_df = pd.read_csv("../weather_data/weather_data.csv", index_col=0)

### 2.ASOS 데이터프레임 전처리

In [118]:
asos_df.head()

,YYMMDDHHMI,STN,WD,WS,GST,GST.1,GST.2,PA,PS,PT,...,TS,TE,TE.1,TE.2,TE.3,ST.1,WH,BF,IR,IX
0,201801010000,108,11,0.5,-9,-9.0,-9,1015.4,1026.4,2,...,-2.2,0.0,0.3,1.0,1.5,-9,-9.0,-9,3.0,2.0
1,201801010100,108,36,0.7,-9,-9.0,-9,1015.1,1026.1,-9,...,-2.7,-0.1,0.3,0.9,1.5,-9,-9.0,-9,3.0,-9.0
2,201801010200,108,27,0.9,-9,-9.0,-9,1015.2,1026.3,-9,...,-3.0,-0.2,0.3,0.9,1.6,-9,-9.0,-9,3.0,-9.0
3,201801010300,108,29,1.0,-9,-9.0,-9,1015.5,1026.6,3,...,-3.5,-0.2,0.3,1.0,1.5,-9,-9.0,-9,3.0,2.0
4,201801010400,108,29,1.1,-9,-9.0,-9,1015.5,1026.6,-9,...,-3.8,-0.3,0.3,0.9,1.6,-9,-9.0,-9,3.0,2.0


#### 2.1 "YYMMDDHHMI" 컬럼을 "Date", "Hour" 컬럼으로 나누기

In [119]:
# "YYMMDDHHMI"컬럼 데이터타입 변환
asos_df["YYMMDDHHMI"] = pd.to_datetime(asos_df["YYMMDDHHMI"], format="%Y%m%d%H%M")

# "Date", "Hour" 컬럼 생성
asos_df["Date"] = asos_df["YYMMDDHHMI"].dt.strftime("%Y-%m-%d")
asos_df["Hour"] = asos_df["YYMMDDHHMI"].dt.strftime("%H")

#### 2.2 필요없는 컬럼 삭제

In [120]:
asos_df = asos_df.drop(['STN', 'GST', 'GST.1', 'GST.2',
                        'PS','PT', 'PR','PV','RN.3',
                        'WC', 'WP', 'WW', 'CA', 'CA.1',
                        'CH', 'CT', 'CT.1', 'CT.2', 'CT.3',
                        'VS', 'SS', 'SI', 'ST', 'TS',
                        'TE', 'TE.1', 'TE.2', 'TE.3', 'ST.1',
                        'WH', 'BF', 'IR', 'IX',], axis=1)

#### 2.3 컬럼 순서 변경

In [121]:
asos_df = asos_df[["YYMMDDHHMI", 'Date', 'Hour', 'WD', 'WS',
                   'PA', 'TA', 'TD', 'HM', 'RN',
                   'RN.1', 'RN.2', 'SD', 'SD.1','SD.2']]

#### 2.4 컬럼명 수정

In [122]:
asos_df = asos_df.rename(columns={"YYMMDDHHMI":"Datetime",
                                  "WD":"Wind Direction",
                                  "WS":"Wind Speed(m/s)",
                                  "PA":"Atmospheric Pressure(hPa)",
                                  "TA":"Temperature(°C)",
                                  "TD":"Dew point Temperature(°C)",
                                  "HM":"Relative Humidity(%)",
                                  "RN":"Rainfall(mm)",
                                  "RN.1":"Daily Rainfall(mm)",
                                  "RN.2":"Daily Rainfall2(mm)",
                                  "SD":"3hr Snowfall(mm)",
                                  "SD.1":"Daily Snowfall(mm)",
                                  "SD.2":"Daily Sonwfall2(mm)"})

#### 2.5 "Date_time", "Date", "Hour" 컬럼에 없는 날짜 추가
- 2019-12-12 21, 22
- 2023-12-11 00 ~ 07
- 2023-12-10 04 ~ 07, 19 ~ 23

In [123]:
# 데이터 생성
columns = asos_df.columns
dates = ["2019-12-12","2023-12-11","2023-12-10"]
hours = [[str(i) for i in range(21,23)],
         [str(i).zfill(2) for i in range(0,8)],
         [str(i).zfill(2) for i in range(4,8)] + [str(i) for i in range(19, 24)]]
default_values = [-9] * (len(columns) - 3)

In [124]:
data = []
for date, hour_list in zip(dates, hours):
    for hour in hour_list:
        date_time = f"{date} {hour}:00:00"
        data.append([date_time, date, hour] + default_values)

In [125]:
df = pd.DataFrame(data, columns=columns)

In [126]:
# asos_df와 합치기
asos_df_fin = pd.concat([asos_df, df], axis=0)

In [127]:
# asos_df_fin 데이터프레임을 생성한 후 "Datetime" 열을 타임스탬프로 변환
asos_df_fin['Datetime'] = pd.to_datetime(asos_df_fin['Datetime'])

# "Datetime" 기준 오름차순 정렬
asos_df_fin.sort_values(by=["Date","Hour"], ascending=True, ignore_index=True, inplace=True)

#### 2.6 데이터프레임을 csv 파일로 저장하기

In [128]:
asos_df_fin.to_csv("../weather_data/asos_data_fin.csv")

### 3.AWS(강수) 데이터프레임 전처리

In [129]:
rf_df = pd.read_csv("../weather_data/rainfall_data.csv", index_col=0)
rf_df.head()

,Date,Hour,108_Daily Rainfall(mm),108_Minute of Daily Rainfall,108_Hourly Rainfall(mm),108_Minute of Hourly Rainfall,108_Accumulated Rainfall(mm) for max 60min,108_Minute of Accumulated Rainfall,112_Daily Rainfall(mm),112_Minute of Daily Rainfall,...,119_Hourly Rainfall(mm),119_Minute of Hourly Rainfall,119_Accumulated Rainfall(mm) for max 60min,119_Minute of Accumulated Rainfall,Average Daily Rainfall(mm),Average Minute of Daily Rainfall,Average Hourly Rainfall(mm),Average Minute of Hourly Rainfall,Average Accumulated Rainfall(mm) for max 60min,Average Minute of Accumulated Rainfall
0,2018-01-01,0,-99.0,-99,-99.0,-59,-99.0,-99,0.1,0,...,0.0,-60,0.0,-59,-32.8,-33.0,-33.0,-59.666667,-33.0,-72.333333
1,2018-01-01,1,0.0,0,0.0,-60,0.0,-2,0.0,0,...,0.0,-60,0.0,-59,0.0,0.0,0.0,-60.000000,0.0,-40.000000
2,2018-01-01,2,0.0,0,0.0,-60,0.0,-59,0.0,0,...,0.0,-60,0.0,-59,0.0,0.0,0.0,-60.000000,0.0,-59.000000
3,2018-01-01,3,0.0,0,0.0,-60,0.0,-59,0.0,0,...,0.0,-60,0.0,-59,0.0,0.0,0.0,-60.000000,0.0,-59.000000
4,2018-01-01,4,0.0,0,0.0,-60,0.0,-59,0.0,0,...,0.0,-60,0.0,-59,0.0,0.0,0.0,-60.000000,0.0,-59.000000


#### 3.1 필요없는 컬럼 삭제

In [ ]:
rf_df.columns[3:20:2] + rf_df.columns[21:]

In [39]:
# 필요없는 컬럼 찾기
drop_cols = rf_df.columns[3::2].to_list() + rf_df.columns[20::2].to_list()

# 필요없는 컬럼 삭제
rf_df.drop(columns=drop_cols, axis=1, inplace=True)


In [42]:
rf_df.head()

,Date,Hour,108_Daily Rainfall(mm),108_Hourly Rainfall(mm),108_Accumulated Rainfall(mm) for max 60min,112_Daily Rainfall(mm),112_Hourly Rainfall(mm),112_Accumulated Rainfall(mm) for max 60min,119_Daily Rainfall(mm),119_Hourly Rainfall(mm),119_Accumulated Rainfall(mm) for max 60min
0,2018-01-01,0,-99.0,-99.0,-99.0,0.1,0.0,0.0,0.5,0.0,0.0
1,2018-01-01,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-01-01,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-01-01,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-01-01,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
rf_df.to_csv("../weather_data/rainfall_data_fin.csv")

### 4.AWS(기온) 데이터프레임 전처리

In [2]:
import pandas as pd

#### 4.1 데이터 불러오기

In [136]:
temp_108_df = pd.read_csv("../weather_data/temperature_data(108).csv", index_col=0)
temp_112_df = pd.read_csv("../weather_data/temperature_data(112).csv", index_col=0)
temp_119_df = pd.read_csv("../weather_data/temperature_data(119).csv", index_col=0)

In [94]:
# 중복 행 제거
temp_108_df.drop_duplicates(inplace=True, ignore_index=True)
temp_112_df.drop_duplicates(inplace=True, ignore_index=True)
temp_119_df.drop_duplicates(inplace=True, ignore_index=True)

#### 4.2 컬럼 순서 변경하기

In [138]:
temp_108_df.columns = ['Datetime', 'ID', 'C', 'mi', 'C.1', 'mi.1', 'C.2', 'mi.2', 'C.3', 'mi.3', '#']
temp_112_df.columns = ['KST', 'ID', 'C', 'mi', 'C.1', 'mi.1', 'C.2', 'mi.2', 'C.3', 'mi.3', '#']
temp_119_df.columns = ['KST', 'ID', 'C', 'mi', 'C.1', 'mi.1', 'C.2', 'mi.2', 'C.3', 'mi.3', '#']

In [143]:
# "Datetime"컬럼 데이터타입 변환
temp_108_df["Datetime"] = pd.to_datetime(temp_108_df["Datetime"], format="%Y%m%d%H%M")

# "Date", "Hour" 컬럼 생성
temp_108_df["Date"] = temp_108_df["Datetime"].dt.strftime("%Y-%m-%d")
temp_108_df["Hour"] = temp_108_df["Datetime"].dt.strftime("%H")

#### 4.3 필요없는 컬럼 삭제

In [144]:
temp_108_df.drop(['ID','mi','mi.1','mi.2','mi.3', '#'], axis=1, inplace=True)
temp_112_df.drop(["KST",'ID','mi','mi.1','mi.2','mi.3', '#'], axis=1, inplace=True)
temp_119_df.drop(["KST",'ID','mi','mi.1','mi.2','mi.3', '#'], axis=1, inplace=True)

#### 4.4 컬럼명 변경하기

In [145]:
temp_108_df.rename(columns={'C':"108_Observed temperature(°C)",
                            'C.1':"108_Average temperature per minute(°C)",
                            'C.2':"108_Highest temperature per minute(°C)",
                            'C.3':"108_Lowest temperature per minute(°C)"},
                    inplace=True)
temp_112_df.rename(columns={'C':"112_Observed temperature(°C)",
                            'C.1':"112_Average temperature per minute(°C)",
                            'C.2':"112_Highest temperature per minute(°C)",
                            'C.3':"112_Lowest temperature per minute(°C)"},
                    inplace=True)
temp_119_df.rename(columns={'C':"119_Observed temperature(°C)",
                            'C.1':"119_Average temperature per minute(°C)",
                            'C.2':"119_Highest temperature per minute(°C)",
                            'C.3':"119_Lowest temperature per minute(°C)"},
                    inplace=True)

#### 4.5 108,112,119 데이터프레임 합치기

In [146]:
temp_df = pd.concat([temp_108_df,temp_112_df,temp_119_df], axis=1)

#### 4.6 "Datetime" 컬럼 오름차순 정렬후, 인덱스 재설정하기

In [148]:
# "Datetime" 컬럼 오름차순 정렬
temp_df.sort_values(by="Datetime", ignore_index=True, inplace=True)
# 인덱스 재설정
temp_df.reset_index(drop=True, inplace=True)

In [151]:
# "Datetime", "Date", "Hour" 순서로 변경
temp_df = temp_df[['Datetime', 'Date', 'Hour',
                           '108_Observed temperature(°C)',
                            '108_Average temperature per minute(°C)',
                            '108_Highest temperature per minute(°C)',
                            '108_Lowest temperature per minute(°C)',
                            '112_Observed temperature(°C)',
                            '112_Average temperature per minute(°C)',
                            '112_Highest temperature per minute(°C)',
                            '112_Lowest temperature per minute(°C)', '119_Observed temperature(°C)',
                            '119_Average temperature per minute(°C)',
                            '119_Highest temperature per minute(°C)',
                            '119_Lowest temperature per minute(°C)']]

#### 4.7 2018년 부터 2023년까지 데이터만 남기기

In [152]:
temp_df = temp_df[temp_df["Date"] < "2024-01-01"]

#### 4.8 데이터프레임을 csv 파일로 저장하기

In [156]:
temp_df.to_csv("../weather_data/temperature_data_fin.csv")

### 5.AWS(바람) 데이터프레임 전처리

#### 5.1 데이터 불러오기

In [46]:
wind_108_df = pd.read_csv("../weather_data/wind_data(108).csv", index_col=0,low_memory=False)
wind_112_df = pd.read_csv("../weather_data/wind_data(112).csv", index_col=0,low_memory=False)
wind_119_df = pd.read_csv("../weather_data/wind_data(119).csv", index_col=0,low_memory=False)

#### 5.2 컬럼 순서 변경하기

In [47]:
wind_108_df.columns = ['KST', 'ID', 'deg', 'm/s', 'mi', 
                       'deg.1', 'm/s.1', 'mi.1', 'mi.2', 'm/s.2',
                       'deg.2', 'm/s.3', 'mi.3', 'mi.4', 'deg.3',
                       'm/s.4', 'mi.5', 'mi.6', '#']
wind_112_df.columns = ['KST', 'ID', 'deg', 'm/s', 'mi', 
                       'deg.1', 'm/s.1', 'mi.1', 'mi.2', 'm/s.2',
                       'deg.2', 'm/s.3', 'mi.3', 'mi.4', 'deg.3',
                       'm/s.4', 'mi.5', 'mi.6', '#']
wind_119_df.columns = ['KST', 'ID', 'deg', 'm/s', 'mi', 
                       'deg.1', 'm/s.1', 'mi.1', 'mi.2', 'm/s.2',
                       'deg.2', 'm/s.3', 'mi.3', 'mi.4', 'deg.3',
                       'm/s.4', 'mi.5', 'mi.6', '#']

#### 5.3 필요없는 컬럼 삭제

In [48]:
wind_108_df.drop(['ID',  'mi', 'deg.1', 'm/s.1',
                  'mi.1', 'mi.2', 'mi.3', 'mi.4', 'deg.3',
                  'm/s.4', 'mi.5', 'mi.6', '#'], axis=1, inplace=True)
wind_112_df.drop(['KST', 'ID',  'mi', 'deg.1', 'm/s.1',
                  'mi.1', 'mi.2', 'mi.3', 'mi.4', 'deg.3',
                  'm/s.4', 'mi.5', 'mi.6', '#'], axis=1, inplace=True)
wind_119_df.drop(['KST', 'ID',  'mi', 'deg.1', 'm/s.1',
                  'mi.1', 'mi.2', 'mi.3', 'mi.4', 'deg.3',
                  'm/s.4', 'mi.5', 'mi.6', '#'], axis=1, inplace=True)

#### 5.4 컬럼명 변경하기

In [49]:
wind_108_df.rename(columns={'KST':"Datetime",
                            'deg':"108_Observed wind direction(deg)",
                            'm/s':"108_Observed wind speed(m/s)",
                            'm/s.2':"108_Average wind speed per minute(m/s)",
                            'deg.2':"108_Maximum wind direction per minute(deg)",
                            'm/s.3':"108_Maximum wind speed per minute(m/s)"},
                    inplace=True)
wind_112_df.rename(columns={'deg':"112_Observed wind direction(deg)",
                            'm/s':"112_Observed wind speed(m/s)",
                            'm/s.2':"112_Average wind speed per minute(m/s)",
                            'deg.2':"112_Maximum wind direction per minute(deg)",
                            'm/s.3':"112_Maximum wind speed per minute(m/s)"},
                    inplace=True)
wind_119_df.rename(columns={'deg':"119_Observed wind direction(deg)",
                            'm/s':"119_Observed wind speed(m/s)",
                            'm/s.2':"119_Average wind speed per minute(m/s)",
                            'deg.2':"119_Maximum wind direction per minute(deg)",
                            'm/s.3':"119_Maximum wind speed per minute(m/s)"},
                    inplace=True)

#### 5.5 108,112,119 데이터프레임 합치기

In [50]:
wind_df = pd.concat([wind_108_df,wind_112_df,wind_119_df], axis=1)

In [52]:
wind_df = wind_df[:-1]

#### 5.6 2018년부터 2023년까지 데이터만 남기기

In [53]:
# "Datetime"컬럼 데이터타입 변환
wind_df["Datetime"] = pd.to_datetime(wind_df["Datetime"], format="%Y%m%d%H%M")

# "Date", "Hour" 컬럼 생성
wind_df["Date"] = wind_df["Datetime"].dt.strftime("%Y-%m-%d")
wind_df["Hour"] = wind_df["Datetime"].dt.strftime("%H")

/var/folders/cy/sw8s66dd02l8b9dj7pqk2ljm0000gn/T/ipykernel_26897/4146877941.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_df["Datetime"] = pd.to_datetime(wind_df["Datetime"], format="%Y%m%d%H%M")
/var/folders/cy/sw8s66dd02l8b9dj7pqk2ljm0000gn/T/ipykernel_26897/4146877941.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_df["Date"] = wind_df["Datetime"].dt.strftime("%Y-%m-%d")
/var/folders/cy/sw8s66dd02l8b9dj7pqk2ljm0000gn/T/ipykernel_26897/4146877941.py:6: SettingWithCopyWarning: 
A value 

#### 5.7 "Datetime" 컬럼 오름차순 정렬후, 인덱스 재설정하기

In [54]:
# "Datetime" 컬럼 오름차순 정렬
wind_df.sort_values(by="Datetime", ignore_index=True, inplace=True)
# 인덱스 재설정
wind_df.reset_index(drop=True, inplace=True)

/var/folders/cy/sw8s66dd02l8b9dj7pqk2ljm0000gn/T/ipykernel_26897/2049578969.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_df.sort_values(by="Datetime", ignore_index=True, inplace=True)


In [55]:
# 컬럼 순서 "Datetime","Date","Hour" 순으로 변경
wind_df = wind_df[['Datetime', 'Date', 'Hour', '108_Observed wind direction(deg)',
                    '108_Observed wind speed(m/s)',
                    '108_Average wind speed per minute(m/s)',
                    '108_Maximum wind direction per minute(deg)',
                    '108_Maximum wind speed per minute(m/s)',
                    '112_Observed wind direction(deg)', '112_Observed wind speed(m/s)',
                    '112_Average wind speed per minute(m/s)',
                    '112_Maximum wind direction per minute(deg)',
                    '112_Maximum wind speed per minute(m/s)',
                    '119_Observed wind direction(deg)', '119_Observed wind speed(m/s)',
                    '119_Average wind speed per minute(m/s)',
                    '119_Maximum wind direction per minute(deg)',
                    '119_Maximum wind speed per minute(m/s)']]

In [56]:
wind_df = wind_df[wind_df["Date"] < "2024-01-01"]

#### 5.8 데이터프레임을 csv 파일로 저장하기

In [58]:
wind_df.to_csv("../weather_data/wind_data_fin.csv")

### 6.AWS(습도) 데이터프레임 전처리

#### 6.1 데이터 불러오기

In [59]:
hum_108_df = pd.read_csv("../weather_data/humidity_data(108).csv", index_col=0)
hum_112_df = pd.read_csv("../weather_data/humidity_data(112).csv", index_col=0)
hum_119_df = pd.read_csv("../weather_data/humidity_data(119).csv", index_col=0)

#### 6.2 컬럼 순서 변경하기

In [60]:
hum_108_df.columns = ['KST', 'ID', '%', 'mi', '%.1', 'mi.1', '%.2', 'mi.2', '%.3', 'mi.3', '#']
hum_112_df.columns = ['KST', 'ID', '%', 'mi', '%.1', 'mi.1', '%.2', 'mi.2', '%.3', 'mi.3', '#']
hum_119_df.columns = ['KST', 'ID', '%', 'mi', '%.1', 'mi.1', '%.2', 'mi.2', '%.3', 'mi.3', '#']

#### 6.3 필요없는 컬럼 삭제

In [61]:
hum_108_df.drop(['ID','mi','mi.1','mi.2','mi.3', '#'], axis=1, inplace=True)
hum_112_df.drop(["KST",'ID','mi','mi.1','mi.2','mi.3', '#'], axis=1, inplace=True)
hum_119_df.drop(["KST",'ID','mi','mi.1','mi.2','mi.3', '#'], axis=1, inplace=True)

#### 6.4 컬럼명 변경하기

In [62]:
hum_108_df.rename(columns={'KST':"Datetime",
                            '%':"108_Observed humidity(%)",
                            '%.1':"108_Average humidity per minute(%)",
                            '%.2':"108_Maximum humidity per minute(%)",
                            '%.3':"108_Minimum humidity per minute(%)"},
                    inplace=True)
hum_112_df.rename(columns={'%':"112_Observed humidity(%)",
                            '%.1':"112_Average humidity per minute(%)",
                            '%.2':"112_Maximum humidity per minute(%)",
                            '%.3':"112_Minimum humidity per minute(%)"},
                    inplace=True)
hum_119_df.rename(columns={'%':"119_Observed humidity(%)",
                            '%.1':"119_Average humidity per minute(%)",
                            '%.2':"119_Maximum humidity per minute(%)",
                            '%.3':"119_Minimum humidity per minute(%)"},
                    inplace=True)

#### 6.5 108,112,119 데이터프레임 합치기

In [63]:
hum_df = pd.concat([hum_108_df,hum_112_df,hum_119_df], axis=1)

In [65]:
hum_df = hum_df[:-1]

#### 6.6 "Datetime" 컬럼 오름차순 정렬후, 인덱스 재설정하기

In [66]:
# "Datetime" 컬럼 오름차순 정렬
hum_df.sort_values(by="Datetime", ignore_index=True, inplace=True)
# 인덱스 재설정
hum_df.reset_index(drop=True, inplace=True)

/var/folders/cy/sw8s66dd02l8b9dj7pqk2ljm0000gn/T/ipykernel_26897/122438431.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hum_df.sort_values(by="Datetime", ignore_index=True, inplace=True)


In [67]:
# "Datetime"컬럼 데이터타입 변환
hum_df["Datetime"] = pd.to_datetime(hum_df["Datetime"], format="%Y%m%d%H%M")

# "Date", "Hour" 컬럼 생성
hum_df["Date"] = hum_df["Datetime"].dt.strftime("%Y-%m-%d")
hum_df["Hour"] = hum_df["Datetime"].dt.strftime("%H")

/var/folders/cy/sw8s66dd02l8b9dj7pqk2ljm0000gn/T/ipykernel_26897/1914792134.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hum_df["Datetime"] = pd.to_datetime(hum_df["Datetime"], format="%Y%m%d%H%M")
/var/folders/cy/sw8s66dd02l8b9dj7pqk2ljm0000gn/T/ipykernel_26897/1914792134.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hum_df["Date"] = hum_df["Datetime"].dt.strftime("%Y-%m-%d")
/var/folders/cy/sw8s66dd02l8b9dj7pqk2ljm0000gn/T/ipykernel_26897/1914792134.py:6: SettingWithCopyWarning: 
A value is t

In [68]:
# 컬럼 순서 "Datetime","Date","Hour" 순으로 변경
hum_df = hum_df[['Datetime', 'Date', 'Hour',
                   '108_Observed humidity(%)',
                    '108_Average humidity per minute(%)',
                    '108_Maximum humidity per minute(%)',
                    '108_Minimum humidity per minute(%)', 
                    '112_Observed humidity(%)',
                    '112_Average humidity per minute(%)',
                    '112_Maximum humidity per minute(%)',
                    '112_Minimum humidity per minute(%)',
                    '119_Observed humidity(%)',
                    '119_Average humidity per minute(%)',
                    '119_Maximum humidity per minute(%)',
                    '119_Minimum humidity per minute(%)']]

#### 6.7 2018년 부터 2023년까지 데이터 남기기

In [69]:
hum_df = hum_df[hum_df["Date"] < "2024-01-01"]

#### 6.8 데이터프레임을 csv 파일로 저장하기

In [71]:
hum_df.to_csv("../weather_data/humidity_data_fin.csv")

### 7.AWS(기압) 데이터프레임 전처리

#### 7.1 데이터 불러오기

In [3]:
ap_108_df = pd.read_csv("../weather_data/ap_data(108).csv")
ap_112_df = pd.read_csv("../weather_data/ap_data(112).csv")
ap_119_df = pd.read_csv("../weather_data/ap_data(119).csv")

/var/folders/cy/sw8s66dd02l8b9dj7pqk2ljm0000gn/T/ipykernel_26897/676420963.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  ap_108_df = pd.read_csv("../weather_data/ap_data(108).csv")
/var/folders/cy/sw8s66dd02l8b9dj7pqk2ljm0000gn/T/ipykernel_26897/676420963.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  ap_112_df = pd.read_csv("../weather_data/ap_data(112).csv")
/var/folders/cy/sw8s66dd02l8b9dj7pqk2ljm0000gn/T/ipykernel_26897/676420963.py:3: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  ap_119_df = pd.read_csv("../weather_data/ap_data(119).csv")


In [4]:
ap_108_df =ap_108_df[1:-1]
ap_112_df =ap_112_df[1:-1]
ap_119_df =ap_119_df[1:-1]

#### 7.2 필요없는 컬럼 삭제하기

In [7]:
ap_108_df.drop(['Unnamed: 1', 'Unnamed: 3', 'PA', 'PA.2', 'PA.4',
                'PA.5', 'PA.6', 'PS', 'PS.1', 'PS.2',
                'PS.3', 'PS.4', 'PS.5', 'PS.6'], axis=1, inplace=True)
ap_112_df.drop(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 3', 'PA', 'PA.2', 'PA.4',
                'PA.5', 'PA.6', 'PS', 'PS.1', 'PS.2',
                'PS.3', 'PS.4', 'PS.5', 'PS.6'], axis=1, inplace=True)
ap_119_df.drop(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 3', 'PA', 'PA.2', 'PA.4',
                'PA.5', 'PA.6', 'PS', 'PS.1', 'PS.2',
                'PS.3', 'PS.4', 'PS.5', 'PS.6'], axis=1, inplace=True)

#### 7.3 컬럼명 변경하기

In [8]:
ap_108_df.rename(columns={'Unnamed: 0':"Datetime",
                            'Unnamed: 2':"108_Observed atmospheric pressure(hPa)",
                            '#':"108_Average atmospheric pressure(hPa)",
                            'PA.1':"108_Highest atmospheric pressure(hPa)",
                            'PA.3':"108_Minimum atmospheric pressure(hPa)"},
                    inplace=True)
ap_112_df.rename(columns={'Unnamed: 2':"112_Observed atmospheric pressure(hPa)",
                            '#':"112_Average atmospheric pressure(hPa)",
                            'PA.1':"112_Highest atmospheric pressure(hPa)",
                            'PA.3':"112_Minimum atmospheric pressure(hPa)"},
                    inplace=True)
ap_119_df.rename(columns={'Unnamed: 2':"119_Observed atmospheric pressure(hPa)",
                            '#':"119_Average atmospheric pressure(hPa)",
                            'PA.1':"119_Highest atmospheric pressure(hPa)",
                            'PA.3':"119_Minimum atmospheric pressure(hPa)"},
                    inplace=True)

#### 7.4 108,112,119 데이터프레임 합치기

In [9]:
ap_df = pd.concat([ap_108_df,ap_112_df,ap_119_df], axis=1)

#### 7.5 "Datetime" 컬럼에서 "Date", "Hour" 컬럼 생성하기

In [12]:
# "Datetime"컬럼 데이터타입 변환
ap_df["Datetime"] = pd.to_datetime(ap_df["Datetime"], format="%Y%m%d%H%M")

# "Date", "Hour" 컬럼 생성
ap_df["Date"] = ap_df["Datetime"].dt.strftime("%Y-%m-%d")
ap_df["Hour"] = ap_df["Datetime"].dt.strftime("%H")

In [13]:
ap_df = ap_df[['Datetime', 'Date', 'Hour', 
               '108_Observed atmospheric pressure(hPa)',
                '108_Average atmospheric pressure(hPa)',
                '108_Highest atmospheric pressure(hPa)',
                '108_Minimum atmospheric pressure(hPa)',
                '112_Observed atmospheric pressure(hPa)',
                '112_Average atmospheric pressure(hPa)',
                '112_Highest atmospheric pressure(hPa)',
                '112_Minimum atmospheric pressure(hPa)',
                '119_Observed atmospheric pressure(hPa)',
                '119_Average atmospheric pressure(hPa)',
                '119_Highest atmospheric pressure(hPa)',
                '119_Minimum atmospheric pressure(hPa)']]

#### 7.6 "Datetime" 컬럼 오름차순 정렬후, 인덱스 재설정하기

In [14]:
# "Datetime" 컬럼 오름차순 정렬
ap_df.sort_values(by="Datetime", ignore_index=True, inplace=True)
# 인덱스 재설정
ap_df.reset_index(drop=True, inplace=True)

#### 7.7 2018년 부터 2023년까지 데이터만 남기기

In [15]:
ap_df = ap_df[ap_df["Date"] < "2024-01-01"]

#### 7.8 데이터프레임을 csv 파일로 저장하기

In [16]:
ap_df.to_csv("../weather_data/ap_data_fin.csv")